In [3]:
!pip install langchain openai PyPDF2 langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00


In [12]:
import PyPDF2
import json
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
import json
from google.colab import userdata
import re
import time
import openai

# Set up OpenAI API key (Make sure to securely input your API key in Colab)
openai_api_key = userdata.get('OPEN_AI_KEY')

chat_model = ChatOpenAI(model="gpt-4o", temperature=0.2, api_key=openai_api_key)

In [6]:
def extract_text_from_pdf(pdf_path):
    """Extracts and returns the full text from a PDF file."""
    try:
        # Read the PDF file
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            full_text = ""
            for page in reader.pages:
                full_text += page.extract_text() + "\n"
        return full_text
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return ""


In [7]:
def process_exam_with_openai(full_text, chat_model):
    """Processes the extracted text to identify and format MCQs using OpenAI."""
    # Define the system prompt
    system_prompt = """
    You are an advanced AI assistant that extracts and processes multiple-choice questions (MCQs) from text.
    Please extract the following for each identified question:

    1. The question stem.
    2. Four answer options (A, B, C, D).
    3. Metadata including:
       - Topic
       - Subtopic
       - Cognitive Level
       - Expected Solution Time

    Use the following JSON format for each question:
    {
        "question": "<question stem>",
        "options": ["<option A>", "<option B>", "<option C>", "<option D>"],
        "metadata": {
            "topic": "<topic>",
            "subtopic": "<subtopic>",
            "grade_level": "9",
            "cognitive_level": "<Bloom's taxonomy level>",
            "expected_solution_time": "<time in seconds>"
        }
    }
    return only the JSON output with nothing else and ensure that the whole response is a proper json file that can be interpreted.
    """

    # Prepare the message with the extracted text
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Extract and format the MCQs from this text:\n{full_text}")
    ]

    # Get response from OpenAI
    response = chat_model(messages)
    response_content = response.content.strip()
    response_content = re.sub(r'```json\n', '', response_content)
    response_content = re.sub(r'\n```', '', response_content)
    #response_content = ['[',response_content,']']

    # Parse and display the formatted questions
    try:
        formatted_questions = json.loads(response_content)
        return formatted_questions
    except json.JSONDecodeError:
        print("Error: Unable to parse the JSON output from OpenAI.")
        print(f"Response Content: {response_content}")
        return []



In [8]:
def process_outline_with_openai(full_text, chat_model):
    """Processes the extracted text to identify and format MCQs using OpenAI."""
    # Define the system prompt
    system_prompt = """
    You are an advanced AI assistant that extracts and processes Course Outline from text.
    Please extract the following for each identified question:

    1. Main topics
    2. suptopics of the main topics
    3. Learning outcomes and brief content

    **output format**:
     topic:
        -subtopic:
            -learning outcomes:
    """

    # Prepare the message with the extracted text
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Extract and format the MCQs from this text:\n{full_text}")
    ]

    # Get response from OpenAI
    response = chat_model(messages)
    response_content = response.content.strip()
    response_content = re.sub(r'```json\n', '', response_content)
    response_content = re.sub(r'\n```', '', response_content)
    #response_content = ['[',response_content,']']

    return response_content

# Run the processing function and display the results


In [9]:
outline_path = '/content/differential-matter_e.pdf'
exam_path = '/content/calculus-english-session1-2021-prac.pdf'

exam_text = extract_text_from_pdf(exam_path)
outline_text = extract_text_from_pdf(outline_path)

In [13]:
# Run the processing function and display the results
processed_questions = process_exam_with_openai(exam_text, chat_model)

print("\nFormatted Questions Extracted:\n")
processed_questions = json.dumps(processed_questions, indent=4)
print(processed_questions)


processed_outline = process_outline_with_openai(outline_text, chat_model)
print("\nFormatted Outline Extracted:\n")
print(processed_outline)


Formatted Questions Extracted:

[
    {
        "question": "If x = sec y, where y \u2208]\u03c0/2, \u03c0[, then dx/dy = ...",
        "options": [
            "-x\u221a(x^2 - 1)",
            "x\u221a(x^2 - 1)",
            "-x\u221a(x^2 + 1)",
            "x\u221a(x^2 + 1)"
        ],
        "metadata": {
            "topic": "Calculus",
            "subtopic": "Differentiation",
            "grade_level": "9",
            "cognitive_level": "Application",
            "expected_solution_time": "60"
        }
    },
    {
        "question": "lim x\u21921 (e^x - e)/(x - 1) = ...",
        "options": [
            "e",
            "-e",
            "1",
            "-1"
        ],
        "metadata": {
            "topic": "Calculus",
            "subtopic": "Limits",
            "grade_level": "9",
            "cognitive_level": "Analysis",
            "expected_solution_time": "45"
        }
    },
    {
        "question": "If lim x\u21920 (ln(x + 1))^k/x = 4, then k = ...",
    

In [14]:
client = openai.OpenAI(api_key = userdata.get('OPEN_AI_KEY') )
def make_api_call(messages, max_tokens, is_final_answer=False):
    for attempt in range(3):
        try:
            response = client.chat.completions.create(
                model="gpt-4o",  # Using GPT-4, adjust as needed
                messages=messages,
                #max_tokens=max_tokens,
                temperature=0.5
            )
            return json.loads(response.choices[0].message.content)
        except json.JSONDecodeError:
            # If JSON parsing fails, return the raw content
            return {"title": "Parsing Error", "content": response.choices[0].message.content, "next_action": "final_answer"}
        except Exception as e:
            if attempt == 2:
                if is_final_answer:
                    return {"title": "Error",
                            "content": f"Failed to generate final answer after 3 attempts. Error: {str(e)}"}
                else:
                    return {"title": "Error", "content": f"Failed to generate step after 3 attempts. Error: {str(e)}",
                            "next_action": "final_answer"}
            time.sleep(1)  # Wait for 1 second before retrying

def generate_response1(prompt):
    messages = [
        {"role": "system", "content": """You are an expert AI assistant with advanced reasoning capabilities. Your task is to provide detailed, step-by-step explanations of your thought process. For each step:

1. Provide a clear, concise title describing the current reasoning phase.
2. Elaborate on your thought process in the content section.
3. Decide whether to continue reasoning or provide a final answer.

Response Format:
Use JSON with keys: 'title', 'content', 'next_action' (values: 'continue' or 'final_answer')

Key Instructions:
- Employ at least 5 distinct reasoning steps.
- Acknowledge your limitations as an AI and explicitly state what you can and cannot do.
- Actively explore and evaluate alternative answers or approaches.
- Critically assess your own reasoning; identify potential flaws or biases.
- When re-examining, employ a fundamentally different approach or perspective.
- Utilize at least 3 diverse methods to derive or verify your answer.
- Incorporate relevant domain knowledge and best practices in your reasoning.
- Quantify certainty levels for each step and the final conclusion when applicable.
- Consider potential edge cases or exceptions to your reasoning.
- Provide clear justifications for eliminating alternative hypotheses.
"""},
        {"role": "user", "content": prompt},
        {"role": "assistant",
         "content": "Understood. I will now think through this step-by-step, following the given instructions and starting by decomposing the problem."}
    ]

    steps = []
    step_count = 1
    total_thinking_time = 0

    while True:
        start_time = time.time()
        step_data = make_api_call(messages, 1000)
        end_time = time.time()
        thinking_time = end_time - start_time
        total_thinking_time += thinking_time

        steps.append((f"Step {step_count}: {step_data['title']}", step_data['content'], thinking_time))

        messages.append({"role": "assistant", "content": json.dumps(step_data)})

        if step_data['next_action'] == 'final_answer':
            break

        step_count += 1

        # Print each step
        print(f"\nStep {step_count}: {step_data['title']}")
        print(step_data['content'])
        print(f"Thinking time: {thinking_time:.2f} seconds")

    # Generate final answer
    messages.append({"role": "user", "content": "Please provide the final answer based on your reasoning above."})

    start_time = time.time()
    final_data = make_api_call(messages, 200, is_final_answer=True)
    end_time = time.time()
    thinking_time = end_time - start_time
    total_thinking_time += thinking_time

    steps.append(("Final Answer", final_data['content'], thinking_time))

    # Print final answer
    print("\nFinal Answer:")
    print(final_data['content'])
    print(f"Thinking time: {thinking_time:.2f} seconds")

    print(f"\nTotal thinking time: {total_thinking_time:.2f} seconds")

    return steps, total_thinking_time

In [17]:
def generate_response(prompt):
    # This function would call the desired model to generate a response.
    # Placeholder implementation: replace with your model-specific code or API call.
    # For example, this could be an API call or a LangChain model function call.
    return generate_response1(prompt)[0][-1][1]

def generate_question_stems(course_outline, example_exam=None):
    """
    Step 1: Generate high-quality question stems based on the course outline and example exam.
    """
    prompt = f"""
Create challenging and high-quality 5 question stems based on the following course outline topics and subtopics:
{json.dumps(course_outline, indent=4)}.

- Ensure questions are clear, concise, and target critical concepts in each subtopic.
- Use the example exam as a style reference to match the difficulty and question type, if available.
- Include a variety of question types (definition, calculation, application, and reasoning) to cover different cognitive levels.

### Output format:
  output should be regular text without any formats
  -Question <question number >: <question stem>

"""

    if example_exam:
        prompt += f"\nExample exam for reference:\n{json.dumps(example_exam, indent=4)}"

    response = generate_response(prompt)
    response_content = response.strip()
    # response_content = re.sub(r'```json\n', '', response_content)
    # response_content = re.sub(r'\n```', '', response_content)
    # Parse the response and extract question stems
    # Simulate generating question stems based on the course outline
    #question_stems = [{"question": f"Sample Question Stem {i+1} on {list(course_outline.keys())[i % len(course_outline)]}"} for i in range(5)]
    return response_content

def generate_answer_options(question_stems, example_exam=None):
    """
    Step 2: Generate effective answer options and strong distractors for each question stem.
    """

    prompt = f"""
Create four answer options (A, B, C, D) for the following questions stems:
{question_stems}

**Requirements:**
- One correct answer and three high-quality distractors.
- Distractors should be plausible and designed to reveal common misconceptions or errors.
- Vary the difficulty of the options: at least one should be very close to the correct answer, and others should include common mistakes.
- Use the example exam as a reference for the complexity and style of options, if provided.

### Output format:
output should be regular text without any formats
  -Question <question number >: <question stem>
  -Option A: <option A>
  -Option B: <option B>
  -Option C: <option C>
  -Option D: <option D>
  -Correct Answer: <correct answer>
"""
    question_stems = generate_response(prompt)
    return question_stems

def review_and_improve(questions):
    """
    Step 3: Review and refine questions for clarity, quality, and appropriate difficulty.
    """
    prompt = f"""
Review the following questions for clarity, quality, and difficulty. Suggest improvements or refinements for each question:

{questions}

**Refinement Criteria:**
- Ensure that the question stem and options are free from ambiguity.
- Balance the difficulty: identify if any options are too obvious or too confusing.
- Improve the distractors if they do not adequately challenge the understanding of the topic.
- Adjust the language and format to ensure they align with a high-quality exam standard.

### Output format:
output should be regular text without any formats
  -Question <question number >: <question stem>
  -Option A: <option A>
  -Option B: <option B>
  -Option C: <option C>
  -Option D: <option D>
  -Correct Answer: <correct answer>
"""

    response = generate_response(prompt)

    # Example refinement logic (simulation)


    return response

def add_metadata(questions, course_outline, grade_level="Grade 9"):
    """
    Step 4: Add meaningful metadata for each question, reflecting cognitive complexity and expected solution time.
    """
    prompt = f"""
Analyze the following multiple-choice questions and extract detailed metadata for each one. Ensure the metadata captures the cognitive complexity, educational objectives, topic relevance, and expected solution time.

**Instructions:**
1. **Cognitive Level**: Determine the cognitive level of the question using Bloom’s Taxonomy (e.g., Knowledge, Comprehension, Application, Analysis, Synthesis, Evaluation). Base this on the type of thinking required to solve the question.
2. **Difficulty Level**: Classify the difficulty as one of the following:
   - **Easy**: Basic recall or simple calculations.
   - **Medium**: Requires reasoning or understanding of the concept.
   - **Hard**: Involves multi-step problem solving, advanced reasoning, or complex analysis.
3. **Topic and Subtopic**: Identify the specific topic and subtopic being tested (e.g., Topic: Algebra, Subtopic: Solving Quadratic Equations).
4. **Grade Level**: Specify the appropriate grade level for this question (e.g., Grade 7, Grade 9).
5. **Expected Solution Time**: Estimate the time required for a typical student to solve this question (e.g., “2 minutes”).
6. **Question Type**: Classify the type of question (e.g., Conceptual, Calculation, Application, Word Problem).
7. **Educational Objective**: Briefly describe the learning objective this question aims to assess (e.g., “Test understanding of the relationship between complementary angles.”).
8. **Misconception Identification**: If applicable, highlight any misconceptions that the distractors are designed to address (e.g., “Distractor A targets a common error in simplifying expressions with negative exponents.”).

**Input Questions:**
{questions}

**Output Format:**
output should be regular text without any formats (no json no html don't create any formats)
you must output with the following format:
-Question <question number >: <question stem>
  -Option A: <option A>
  -Option B: <option B>
  -Option C: <option C>
  -Option D: <option D>
  -Correct Answer: <correct answer>
  -Metadata:
    -Cognitive Level: <cognitive level>
    -Difficulty Level: <difficulty level>
    -Topic: <topic>
    -Subtopic: <subtopic>
    -Grade Level: <grade level>
    -Expected Solution Time: <expected solution time>

"""
    questions = generate_response(prompt)
    return questions




In [18]:
# Step-by-step question generation with enhanced prompts
question_stems = generate_question_stems(processed_outline, processed_questions)
#print(question_stems)
questions_with_options = generate_answer_options(question_stems, processed_questions)
#print(questions_with_options)
refined_questions = review_and_improve(questions_with_options)
#print(refined_questions)
questions_with_metadata = add_metadata(refined_questions, processed_outline)


Step 2: Decomposing the Task
The task requires analyzing multiple-choice questions to extract detailed metadata. This involves evaluating cognitive complexity, educational objectives, topic relevance, and expected solution time for each question. The metadata needs to include various educational dimensions, such as cognitive level, difficulty, topic, subtopic, grade level, expected solution time, question type, educational objective, and misconception identification.
Thinking time: 1.10 seconds

Step 3: Analyzing Question 1
Question 1 involves finding the derivative of a function involving inverse trigonometric functions. The cognitive level required is 'Analysis' as it involves differentiating a combination of inverse trigonometric functions, which requires understanding of calculus concepts. The difficulty level is 'Hard' due to the complexity of working with inverse trigonometric derivatives and the need for careful manipulation. The topic is 'Calculus' and the subtopic is 'Differe